In [1]:
import tensorflow as tf
import os
import zipfile

In [19]:
Desired_accuracy = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip",'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy')>Desired_accuracy):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2020-07-26 09:35:33--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 64.233.188.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2020-07-26 09:35:33 (84.5 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [20]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import  RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator  = train_datagen.flow_from_directory(
    "tmp/h-or-s",
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [21]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/15
8/8 [==============================] - 2s 189ms/step - loss: 1.1914 - accuracy: 0.5625
Epoch 2/15
8/8 [==============================] - 2s 192ms/step - loss: 0.5276 - accuracy: 0.6750
Epoch 3/15
8/8 [==============================] - 2s 191ms/step - loss: 0.1924 - accuracy: 0.9500
Epoch 4/15
8/8 [==============================] - 1s 187ms/step - loss: 0.1925 - accuracy: 0.9500
Epoch 5/15
8/8 [==============================] - 2s 189ms/step - loss: 0.0741 - accuracy: 0.9875
Epoch 6/15
8/8 [==============================] - 2s 188ms/step - loss: 0.1672 - accuracy: 0.8875
Epoch 7/15
8/8 [==============================] - 2s 190ms/step - loss: 0.0564 - accuracy: 0.9750
Epoch 8/15
8/8 [==============================] - 2s 188ms/step - loss: 0.0463 - accuracy: 0.9875
Epoch 9/15
8/8 [==============================] - ETA: 0s - loss: 0.0178 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 2s 191ms/step - loss: 0.0178 - accurac